In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
plt.style.use('ggplot')

list of columns to drop:

In [18]:
x = pd.read_csv('train_values.csv', index_col='row_id')
y = pd.read_csv('train_labels.csv', index_col='row_id')
filtered = pd.read_csv('repayment_rates_filtered.csv').transpose()

In [36]:
features = list(filtered.iloc[1:200,0].index)

In [37]:
features

['student__share_lowincome_0_30000',
 'student__demographics_avg_family_income',
 'student__demographics_median_family_income',
 'student__avg_dependent_income_2014dollars',
 'student__students_with_pell_grant',
 'student__share_highincome_75001_110000',
 'student__share_highincome_110001plus',
 'student__share_middleincome_48001_75000',
 'student__share_firstgeneration_parents_somecollege',
 'student__share_dependent_lowincome_0_300000',
 'student__demographics_first_generation',
 'aid__pell_grant_rate',
 'student__demographics_dependent',
 'school__ownership-Private for-profit',
 'student__fafsa_sent_5plus_college_allyrs',
 'student__share_firstgeneration_parents_highschool',
 'student__fafsa_sent_4_college_allyrs',
 'student__demographics_age_entry',
 'student__fafsa_sent_3_college_allyrs',
 "school__degrees_awarded_predominant-Predominantly bachelor's-degree granting",
 'academics__program_bachelors_biological',
 'academics__program_bachelors_mathematics',
 'academics__program_bach

feature engineering:

In [ ]:
def num_bach(x):
    return x.filter(like='_bach').replace(0,np.nan).sum(axis=1)
x['num_bach'] = num_bach(x)

In [ ]:
drop_cols = []
drop_cols.extend(list(x.filter(like='academics__program_assoc')))
drop_cols.extend(list(x.filter(like='academics__program_bach')))
drop_cols.extend(list(x.filter(like='academics__program_cert')))
drop_cols.extend(list(x.filter(like='completion__')))
drop_cols.extend(list(x.filter(like='_program_percentage_')))
drop_cols.extend(list(x.filter(like='cost__')))
drop_cols.extend(list(x.filter(like='_debt_number_')))
drop_cols.extend(list(x.filter(like='cumulative_debt_')))
drop_cols.extend(list(x.filter(like='median_debt_completers_')))
drop_cols.extend(list(x.filter(like='student__')))
drop_cols.extend(['report_year'])

linear = []
temp = list(x.filter(like='_program_percentage_'))
linear.extend([temp[i] for i in ( 6, 9, 10, 11, 14, 15, 16, 33, 34)])

temp = list(x.filter(like='completion__'))
linear.extend([temp[i] for i in (6, 9, 10, 11, 13, 14, 15, 16, 17)])

temp = list(x.filter(like='cost__'))
linear.extend([temp[i] for i in ( 0, 3, 14, 18, 19, 25, 28, 30, 31, 32, 33)])

temp = list(x.filter(like='student__'))
linear.extend([temp[i] for i in (1, 2, 3, 4, 5, 6, 8, 9, 10, 14, 17, 23, 24, 25, 26, 31, 32, 33, 36, 37, 38, 40, 41, 42, 
                                 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 56)])

linear.extend(['school_faculty_salary', 'school_ft_faculty_rate', ])

drop_cols = [x for x in drop_cols if x not in linear]

In [ ]:
def replace_01(x, inplace=True):
    x['academics__program_percentage_computer'].replace([1], 0.1, inplace=inplace)
    x['academics__program_percentage_health'].replace([0,1], np.nan, inplace=inplace)
    x['academics__program_percentage_humanities'].replace([0,1], np.nan, inplace=inplace)
    x['completion__completion_rate_4yr_150_asian'].replace([0,1], np.nan, inplace=inplace)
    x['completion__completion_rate_4yr_150_black'].replace([0,1], np.nan, inplace=inplace)
    x['completion__completion_rate_4yr_150_hispanic'].replace([0,1], np.nan, inplace=inplace)
    x['completion__completion_rate_4yr_150_nonresident_alien'].replace([0,1], np.nan, inplace=inplace)
    x['completion__completion_rate_4yr_150_race_unknown'].replace([0,1], np.nan, inplace=inplace)
    x['completion__completion_rate_4yr_150_white'].replace([0,1], np.nan, inplace=inplace)
    x['completion__completion_rate_4yr_100nt'].replace([1], 0.2, inplace=inplace)
    x['completion__completion_rate_4yr_150nt'].replace([1], 0.5, inplace=inplace)
    x['school__ft_faculty_rate'].replace([1], np.nan, inplace=inplace)
    x['student__part_time_share'].replace([1],0.2, inplace=inplace)    
    x['student__retention_rate_four_year_full_time'].replace([0,1],[0.3, np.nan], inplace=inplace)    
    x['student__retention_rate_four_year_part_time'].replace([0,1], np.nan, inplace=inplace)
    x['student__share_first_time_full_time'].replace([1],[np.nan], inplace=inplace)
    
    
replace_01(x)

In [ ]:
log_columns = ['academics__program_percentage_engineering',
          'academics__program_percentage_english',
          'academics__program_percentage_history',
          'academics__program_percentage_social_science',
          'cost__title_iv_private_by_income_level_75001_110000',
          'cost__title_iv_public_by_income_level_110001_plus']

def get_log(x, log_columns=log_columns, inplace=True):
    for col in log_columns:
        x[col] = x[col].replace(0,np.nan).apply(np.log)

get_log(x)

In [ ]:
school_categoricals = ['school__carnegie_basic',
               'school__carnegie_size_setting',
               'school__carnegie_undergrad',
               'school__institutional_characteristics_level',
               'school__locale',
               'school__main_campus',
               'school__men_only',
               'school__minority_serving_aanipi',
               'school__minority_serving_hispanic',
               'school__minority_serving_historically_black',
               'school__minority_serving_nant',
               'school__minority_serving_predominantly_black',
               'school__minority_serving_tribal',
               'school__online_only',
               'school__religious_affiliation',
               'school__state',
               'school__ownership',
               'school__degrees_awarded_highest',
               'school__degrees_awarded_predominant',
               'school__region_id',
               'school__women_only',
               'school__minority_serving_annh',
               ]
for cat in school_categoricals:
    x[cat] = x[cat].astype('category')

In [ ]:
x.drop(drop_cols, axis=1, inplace=True)

In [ ]:
x.columns[x.dtypes == 'category']

In [ ]:
cat_cols = ['school__carnegie_basic', 'school__carnegie_size_setting',
       'school__carnegie_undergrad', 'school__degrees_awarded_highest',
       'school__degrees_awarded_predominant',
       'school__institutional_characteristics_level', 'school__locale',
       'school__main_campus', 'school__men_only',
       'school__minority_serving_aanipi', 'school__minority_serving_annh',
       'school__minority_serving_hispanic',
       'school__minority_serving_historically_black',
       'school__minority_serving_nant',
       'school__minority_serving_predominantly_black',
       'school__minority_serving_tribal', 'school__online_only',
       'school__ownership', 'school__region_id',
       'school__religious_affiliation', 'school__state', 'school__women_only']

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, Imputer, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error

              

## Random Forest Regressor: Categories

In [ ]:
x_cat = pd.get_dummies(x, sparse=True)
x_num = x.drop(cat_cols, axis=1)

In [ ]:
data = x_cat

X_train, X_test, y_train, y_test = train_test_split(data, y['repayment_rate'], random_state = 42)  

pipe_forest = make_pipeline(Imputer(strategy='median'), RandomForestRegressor(n_estimators=20, 
                                                                              max_features=200, 
                                                                              n_jobs=-1,
                                                                              random_state=42,
                                                                              verbose=False))
pipe_forest.fit(X_train, y_train)

param_grid = {}

grid = GridSearchCV(pipe_forest, param_grid=param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

print('Best CV Accuracy: {:.3f}'.format(grid.best_score_))
print('Test Set Score: {:.3f}'.format(grid.score(X_test, y_test)))
print('Best_Parameters: {}'.format(grid.best_params_))
print('RMSE: {}'.format(mean_squared_error(y_test, grid.predict(X_test))))

## Random Forest Regressor: no categories

In [ ]:
data = x_num

X_train, X_test, y_train, y_test = train_test_split(data, y['repayment_rate'], random_state = 42)

pipe_forest = make_pipeline(Imputer(strategy='median'), RandomForestRegressor(n_estimators=20, 
                                                                              max_features=50, 
                                                                              n_jobs=-1,
                                                                              random_state=42,
                                                                              verbose=False))
pipe_forest.fit(X_train, y_train)

param_grid = {}

grid = GridSearchCV(pipe_forest, param_grid=param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

print('Best CV Accuracy: {:.3f}'.format(grid.best_score_))
print('Test Set Score: {:.3f}'.format(grid.score(X_test, y_test)))
print('Best_Parameters: {}'.format(grid.best_params_))
print('RMSE: {}'.format(mean_squared_error(y_test, grid.predict(X_test))))

## Regression: Categories

In [ ]:
data = x_cat

X_train, X_test, y_train, y_test = train_test_split(data, y['repayment_rate'], random_state = 42) 
pipe = Pipeline([('preprocessing',Imputer(strategy='median')), ('regressor', LinearRegression())])
                      
pipe.fit(X_train, y_train)

param_grid = [
    {'regressor': [LinearRegression()]},
    {'regressor': [Lasso()]},
    {'regressor': [Ridge()]},
]

grid = GridSearchCV(pipe, param_grid=param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

print('Best CV Accuracy: {:.3f}'.format(grid.best_score_))
print('Test Set Score: {:.3f}'.format(grid.score(X_test, y_test)))
print('Best_Parameters: {}'.format(grid.best_params_))
print('RMSE: {}'.format(mean_squared_error(y_test, grid.predict(X_test))))

## Regression: No Categories

In [ ]:
data = x_num

X_train, X_test, y_train, y_test = train_test_split(data, y['repayment_rate'], random_state = 42) 
pipe = Pipeline([('preprocessing',Imputer(strategy='median')), ('regressor', LinearRegression())])
                      
pipe.fit(X_train, y_train)

param_grid = [
    {'regressor': [LinearRegression()]},
    {'regressor': [Lasso()]},
    {'regressor': [Ridge()]},
]

grid = GridSearchCV(pipe, param_grid=param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

print('Best CV Accuracy: {:.3f}'.format(grid.best_score_))
print('Test Set Score: {:.3f}'.format(grid.score(X_test, y_test)))
print('Best_Parameters: {}'.format(grid.best_params_))
print('RMSE: {}'.format(mean_squared_error(y_test, grid.predict(X_test))))

## KNeighbors Regression: Categories

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

data = x_cat

X_train, X_test, y_train, y_test = train_test_split(data, y['repayment_rate'], random_state = 42)

pipe_forest = make_pipeline(Imputer(strategy='median'), KNeighborsRegressor(n_jobs=-1))
pipe_forest.fit(X_train, y_train)

param_grid = {'kneighborsregressor__n_neighbors':[5, 10, 15, 20]}

grid = GridSearchCV(pipe_forest, param_grid=param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

print('Best CV Accuracy: {:.3f}'.format(grid.best_score_))
print('Test Set Score: {:.3f}'.format(grid.score(X_test, y_test)))
print('Best_Parameters: {}'.format(grid.best_params_))
print('RMSE: {}'.format(mean_squared_error(y_test, grid.predict(X_test))))

## KNeighbors Regression: No categories

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

data = x_num

X_train, X_test, y_train, y_test = train_test_split(data, y['repayment_rate'], random_state = 42)

pipe_forest = make_pipeline(Imputer(strategy='median'), KNeighborsRegressor(n_jobs=-1))
pipe_forest.fit(X_train, y_train)

param_grid = {'kneighborsregressor__n_neighbors':[5, 10, 15, 20]}

grid = GridSearchCV(pipe_forest, param_grid=param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

print('Best CV Accuracy: {:.3f}'.format(grid.best_score_))
print('Test Set Score: {:.3f}'.format(grid.score(X_test, y_test)))
print('Best_Parameters: {}'.format(grid.best_params_))
print('RMSE: {}'.format(mean_squared_error(y_test, grid.predict(X_test))))

## In every case, categories perform better than with no categories. 
## RandomForest performs best.

### Best Performing Model: RandomForestRegressor with Categoricals

In [ ]:
x.head()

In [ ]:
data = x_cat

X_train, X_test, y_train, y_test = train_test_split(data, y['repayment_rate'], random_state = 42)  

pipe_forest = make_pipeline(Imputer(), RandomForestRegressor(n_estimators=10, 
                                                                              #max_features=200, 
                                                                              n_jobs=-1,
                                                                              random_state=42,
                                                                              verbose=False))
pipe_forest.fit(X_train, y_train)

param_grid = {'imputer__strategy': ['mean', 'median', 'most_frequent']}

grid = GridSearchCV(pipe_forest, param_grid=param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

print('Best CV Accuracy: {:.3f}'.format(grid.best_score_))
print('Test Set Score: {:.3f}'.format(grid.score(X_test, y_test)))
print('Best_Parameters: {}'.format(grid.best_params_))
print('RMSE: {}'.format(mean_squared_error(y_test, grid.predict(X_test))))

In [ ]:
imp = Imputer(strategy='median')
rfg = RandomForestRegressor(n_estimators=10,
                           n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(x_cat, y['repayment_rate'], random_state = 42)  

X_train_imp = imp.fit_transform(X_train)
rfg.fit(X_train_imp, y_train)
X_test_imp = imp.fit_transform(X_test)

print('Train Set Score: {:.3f}'.format(rfg.score(X_train_imp, y_train)))
print('Test Set Score: {:.3f}'.format(rfg.score(X_test_imp, y_test)))
print('RMSE: {}'.format(mean_squared_error(y_test, rfg.predict(X_test_imp))))

In [ ]:
def plot_feature_importance(model):
    n_features = x_cat.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), data.columns) #np.arange(n_features))
    plt.xlabel('feature importance')
    plt.ylabel('feature')
plt.figure(figsize=(15,200))
plt.xlim(0, 0.001)    
plot_feature_importance(rfg)

In [ ]:
from itertools import compress

fil = rfg.feature_importances_ > 0.001
best_features = list(compress(data.columns, fil))

imp = Imputer(strategy='median')
rfg = RandomForestRegressor(n_estimators=30,
                           n_jobs=-1)

data=x_cat

X_train, X_test, y_train, y_test = train_test_split(data.loc[:,best_features], y['repayment_rate'], random_state = 42)  

X_train_imp = imp.fit_transform(X_train)
rfg.fit(X_train_imp, y_train)
X_test_imp = imp.fit_transform(X_test)

print('Train Set Score: {:.3f}'.format(rfg.score(X_train_imp, y_train)))
print('Test Set Score: {:.3f}'.format(rfg.score(X_test_imp, y_test)))
print('RMSE: {}'.format(mean_squared_error(y_test, rfg.predict(X_test_imp))))

In [ ]:
x = pd.read_csv('train_values.csv', index_col='row_id')
x = pd.get_dummies(x, sparse=True)
y = pd.read_csv('train_labels.csv', index_col='row_id')

imp = Imputer(strategy='median')
rfg = RandomForestRegressor(n_estimators=30,
                           n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(x, y['repayment_rate'], random_state = 42)  

X_train_imp = imp.fit_transform(X_train)
rfg.fit(X_train_imp, y_train)
X_test_imp = imp.fit_transform(X_test)

print('Train Set Score: {:.3f}'.format(rfg.score(X_train_imp, y_train)))
print('Test Set Score: {:.3f}'.format(rfg.score(X_test_imp, y_test)))
print('RMSE: {}'.format(mean_squared_error(y_test, rfg.predict(X_test_imp))))

In [ ]:
def plot_feature_importance(model, dataframe):
    n_features = dataframe.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), x.columns) #np.arange(n_features))
    plt.xlabel('feature importance')
    plt.ylabel('feature')
plt.figure(figsize=(15,200))
plt.xlim(0, 0.001)    
plot_feature_importance(rfg, x)

In [ ]:
x = pd.read_csv('train_values.csv', index_col='row_id')
x = pd.get_dummies(x, sparse=True)
y = pd.read_csv('train_labels.csv', index_col='row_id')

X_train, X_test, y_train, y_test = train_test_split(x, y['repayment_rate'], random_state = 42)  

X_train_imp = imp.fit_transform(X_train)

rfg = RandomForestRegressor(n_jobs=-1, max_features=200)
rfg.fit(X_train_imp, y_train)
X_test_imp = imp.fit_transform(X_test)

print('Train Set Score: {:.3f}'.format(rfg.score(X_train_imp, y_train)))
print('Test Set Score: {:.3f}'.format(rfg.score(X_test_imp, y_test)))
print('RMSE: {}'.format(mean_squared_error(y_test, rfg.predict(X_test_imp))))

### Since I'm using decision trees, feature selection doesn't really matter. So I'll start a new notebook based on this.